In [ ]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlayWrightTimeout
import time

In [ ]:
%pip install beautifulsoup4
%pip install playwright

In [ ]:
SEASONS = list(range(2023,2024))

In [ ]:
# Define directory paths
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

# Create directories
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

# Check directories
print("Directories created:")
print(os.listdir(DATA_DIR))

In [ ]:
!playwright install

In [ ]:
async def get_html(url, selector, sleep=2, retries=3): #function with parameters
  html = None
  for i in range(1, retries+1):
    time.sleep(sleep * 1) #pauses program for a few seconds

    try: #for errors
      async with async_playwright() as p:
        browser = await p.chromium.launch() #await turns synchronous
        page = await browser.new_page()
        await page.goto(url)
        print(await page.title())
        html = await page.inner_html(selector)
    except PlayWrightTimeout:
      print(f"Timeout error on {url}")
      continue
    else:
      break
  return html

In [ ]:
async def scrape_season(season):
  url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
  html = await get_html(url, "#content .filter")

  soup = BeautifulSoup(html)
  links = soup.find_all("a")
  href = [l["href"] for l in links] #extract href
  standings_pages = [f"https://basketball-reference.com{l}" for l in href] #href missing end portion of url, so we loop through all and add to url

  for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1]) #save end part of url
    if os.path.exists(save_path): #if exists dont scrape again
      continue

    html = await get_html(url, "#all_schedule")
    with open(save_path, "w+") as f:
        f.write(html)

In [ ]:
for season in SEASONS:
  await scrape_season(season)

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
#Parsing box scores
async def scrape_game(standings_file):
  with open(standings_file, 'r') as f:
    html = f.read()
  soup = BeautifulSoup(html)
  links = soup.find_all("a")
  hrefs = [l.get("href") for l in links]
  box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
  box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

  for url in box_scores:
    save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue
    html = await get_html(url, "#content")
    if not html:
      continue
    with open(save_path, "w+", encoding='utf-8') as f:
        f.write(html)


In [ ]:
standings_files = [s for s in standings_files if ".html" in s]

In [ ]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)